# 🚀 Day 3: GPU Memory Fundamentals

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapati3/cuda-lab/blob/main/learning-path/week-01/day-3-memory-basics.ipynb)

## Learning Philosophy

> **CUDA C++ First, Python/Numba as Optional Backup**

This notebook shows:
1. **CUDA C++ code** - The PRIMARY implementation you should learn
2. **Python/Numba code** - OPTIONAL for quick interactive testing in Colab

> **Note:** If running on Google Colab, go to `Runtime → Change runtime type → T4 GPU` before starting!

---

In [ ]:
# ⚙️ Colab/Local Setup - Run this first!
# Python/Numba is OPTIONAL - for quick interactive testing only
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

import numpy as np
from numba import cuda
import math
import time

print("\n⚠️  Remember: CUDA C++ code is the PRIMARY learning material!")
print("   Python/Numba is provided for quick interactive testing only.")

# Day 3: GPU Memory Fundamentals

Memory management is **the most important skill** in CUDA programming. Today you'll learn:
- GPU memory hierarchy
- Explicit memory management (`cudaMalloc`, `cudaMemcpy`, `cudaFree`)
- Unified/Managed memory (`cudaMallocManaged`)
- Pinned vs pageable memory for transfers
- Performance implications of each approach

---

## 1. The Memory Hierarchy

```
┌─────────────────────────────────────────────────────────────────┐
│                        GPU MEMORY HIERARCHY                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   Registers (per thread)     ← Fastest (~0 latency)             │
│   ├── ~255 32-bit registers per thread                          │
│   └── Private to each thread                                    │
│                                                                  │
│   Shared Memory (per block)  ← Very fast (~5 cycles)            │
│   ├── 48-164 KB per SM                                          │
│   └── Shared between threads in same block                      │
│                                                                  │
│   L1 Cache / Texture Cache   ← Fast (~30 cycles)                │
│   └── Automatic caching                                         │
│                                                                  │
│   L2 Cache                   ← Medium (~200 cycles)             │
│   └── Shared across all SMs                                     │
│                                                                  │
│   Global Memory (VRAM)       ← Slow (~400+ cycles)              │
│   ├── 4-80 GB (main GPU memory)                                 │
│   └── Accessible by all threads                                 │
│                                                                  │
├─────────────────────────────────────────────────────────────────┤
│   Host Memory (System RAM)   ← Very slow (PCIe transfer)        │
│   └── Must be copied to GPU before use                          │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup
import numpy as np
from numba import cuda
import math
import time

print("CUDA device:", cuda.get_current_device().name.decode())

# Get memory info
ctx = cuda.current_context()
free_mem, total_mem = ctx.get_memory_info()
print(f"GPU Memory: {free_mem/1e9:.2f} GB free / {total_mem/1e9:.2f} GB total")

## 2. Explicit Memory Management

The traditional CUDA workflow requires **explicit** memory transfers:

```
1. Allocate memory on GPU (cudaMalloc)
2. Copy data from CPU to GPU (cudaMemcpy H2D)
3. Launch kernel
4. Copy results from GPU to CPU (cudaMemcpy D2H)
5. Free GPU memory (cudaFree)
```

In Numba:
- `cuda.to_device(array)` - Allocates and copies to GPU
- `cuda.device_array(shape, dtype)` - Allocates on GPU only
- `device_array.copy_to_host()` - Copies back to CPU

In [ ]:
@cuda.jit
def saxpy_kernel(a, x, y, out, n):
    """SAXPY: out = a*x + y (Single-precision A*X Plus Y)"""
    idx = cuda.grid(1)
    if idx < n:
        out[idx] = a * x[idx] + y[idx]

def explicit_memory_workflow(n, a=2.0):
    """Demonstrate explicit memory management"""
    print(f"\n📊 Explicit Memory Workflow (N = {n:,})")
    print("-" * 50)
    
    # Step 1: Create data on CPU
    x_host = np.random.randn(n).astype(np.float32)
    y_host = np.random.randn(n).astype(np.float32)
    out_host = np.zeros(n, dtype=np.float32)
    
    # Step 2: Allocate and copy to GPU
    start = time.perf_counter()
    x_device = cuda.to_device(x_host)        # Allocate + copy
    y_device = cuda.to_device(y_host)        # Allocate + copy
    out_device = cuda.device_array(n, dtype=np.float32)  # Allocate only
    h2d_time = time.perf_counter() - start
    print(f"  Host→Device transfer: {h2d_time*1000:.3f} ms")
    
    # Step 3: Launch kernel
    threads = 256
    blocks = math.ceil(n / threads)
    
    start = time.perf_counter()
    saxpy_kernel[blocks, threads](a, x_device, y_device, out_device, n)
    cuda.synchronize()
    kernel_time = time.perf_counter() - start
    print(f"  Kernel execution:      {kernel_time*1000:.3f} ms")
    
    # Step 4: Copy result back
    start = time.perf_counter()
    out_host = out_device.copy_to_host()
    d2h_time = time.perf_counter() - start
    print(f"  Device→Host transfer:  {d2h_time*1000:.3f} ms")
    
    print(f"  Total time:            {(h2d_time + kernel_time + d2h_time)*1000:.3f} ms")
    
    # Verify
    expected = a * x_host + y_host
    print(f"  ✅ Correct: {np.allclose(out_host, expected)}")
    
    return h2d_time, kernel_time, d2h_time

# Run with different sizes
for size in [100_000, 1_000_000, 10_000_000]:
    explicit_memory_workflow(size)

## 3. Unified Memory (cudaMallocManaged)

**Unified Memory** simplifies programming - memory is automatically migrated between CPU and GPU as needed.

```
┌──────────────┐                    ┌──────────────┐
│     CPU      │◄──── Automatic ────►│     GPU      │
│              │      Migration      │              │
│  ptr[i] = x  │                    │  ptr[i] = y  │
└──────────────┘                    └──────────────┘
        Same pointer works on both!
```

**Pros:**
- Simpler code
- No explicit transfers needed
- Can access more data than GPU memory (oversubscription)

**Cons:**
- May have higher latency (page faults)
- Less control over when transfers happen
- Can be slower if not used carefully

In [ ]:
def unified_memory_workflow(n, a=2.0):
    """Demonstrate unified memory (managed memory)"""
    print(f"\n📊 Unified Memory Workflow (N = {n:,})")
    print("-" * 50)
    
    # Allocate managed memory - accessible from both CPU and GPU
    x = cuda.managed_array(n, dtype=np.float32)
    y = cuda.managed_array(n, dtype=np.float32)
    out = cuda.managed_array(n, dtype=np.float32)
    
    # Initialize on CPU (memory will migrate to GPU when needed)
    x[:] = np.random.randn(n).astype(np.float32)
    y[:] = np.random.randn(n).astype(np.float32)
    
    # Launch kernel (data migrates automatically)
    threads = 256
    blocks = math.ceil(n / threads)
    
    start = time.perf_counter()
    saxpy_kernel[blocks, threads](a, x, y, out, n)
    cuda.synchronize()
    total_time = time.perf_counter() - start
    print(f"  Kernel + migration time: {total_time*1000:.3f} ms")
    
    # Access result on CPU (data migrates back automatically)
    expected = a * x + y
    print(f"  ✅ Correct: {np.allclose(out, expected)}")
    
    return total_time

# Compare with explicit management
for size in [100_000, 1_000_000, 10_000_000]:
    unified_memory_workflow(size)

## 4. Pinned (Page-Locked) Memory

Normal CPU memory can be **paged out** to disk by the OS. This causes problems for DMA transfers.

**Pinned memory** is locked in physical RAM - no paging allowed.

```
┌─────────────────────────────────────────────────────────────┐
│  Pageable Memory (default)     │  Pinned Memory             │
├─────────────────────────────────────────────────────────────┤
│  CPU Memory                    │  CPU Memory (locked)       │
│      ↓ (copy to staging)       │      ↓ (direct DMA)        │
│  Pinned Buffer                 │                            │
│      ↓ (DMA to GPU)            │      ↓ (DMA to GPU)        │
│  GPU Memory                    │  GPU Memory                │
├─────────────────────────────────────────────────────────────┤
│  TWO copies!                   │  ONE copy! (faster)        │
│  Can be swapped to disk        │  Always in RAM             │
│  Unlimited size                │  Limited by system RAM     │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
def compare_pinned_vs_pageable(n):
    """Compare transfer speeds with pinned vs pageable memory"""
    print(f"\n📊 Pinned vs Pageable Memory (N = {n:,})")
    print("-" * 50)
    
    # Pageable memory (default NumPy allocation)
    pageable = np.random.randn(n).astype(np.float32)
    
    # Pinned memory
    pinned = cuda.pinned_array(n, dtype=np.float32)
    pinned[:] = pageable  # Copy data to pinned
    
    # Measure pageable transfer
    start = time.perf_counter()
    for _ in range(10):
        d_pageable = cuda.to_device(pageable)
        cuda.synchronize()
    pageable_time = (time.perf_counter() - start) / 10
    
    # Measure pinned transfer
    start = time.perf_counter()
    for _ in range(10):
        d_pinned = cuda.to_device(pinned)
        cuda.synchronize()
    pinned_time = (time.perf_counter() - start) / 10
    
    # Calculate bandwidth
    data_size_gb = n * 4 / 1e9  # float32 = 4 bytes
    pageable_bw = data_size_gb / pageable_time
    pinned_bw = data_size_gb / pinned_time
    
    print(f"  Pageable: {pageable_time*1000:.3f} ms ({pageable_bw:.2f} GB/s)")
    print(f"  Pinned:   {pinned_time*1000:.3f} ms ({pinned_bw:.2f} GB/s)")
    print(f"  Speedup:  {pageable_time/pinned_time:.2f}x")

# Compare for different sizes
for size in [1_000_000, 10_000_000, 50_000_000]:
    compare_pinned_vs_pageable(size)

## 5. When Transfer Time Dominates

For simple operations like vector addition, **memory transfers dominate execution time**.

This is why:
1. GPU is best for **compute-intensive** operations
2. You should **minimize transfers** (keep data on GPU)
3. **Batch operations** together before copying back

In [ ]:
def analyze_transfer_overhead(n):
    """Show how transfer time compares to compute time"""
    print(f"\n📊 Transfer vs Compute Analysis (N = {n:,})")
    print("-" * 60)
    
    a = np.random.randn(n).astype(np.float32)
    b = np.random.randn(n).astype(np.float32)
    
    # Measure H2D transfer
    start = time.perf_counter()
    a_d = cuda.to_device(a)
    b_d = cuda.to_device(b)
    c_d = cuda.device_array(n, dtype=np.float32)
    cuda.synchronize()
    h2d_time = time.perf_counter() - start
    
    # Measure kernel (data already on GPU)
    threads = 256
    blocks = math.ceil(n / threads)
    
    @cuda.jit
    def add_kernel(a, b, c):
        idx = cuda.grid(1)
        if idx < c.size:
            c[idx] = a[idx] + b[idx]
    
    # Warmup
    add_kernel[blocks, threads](a_d, b_d, c_d)
    cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(100):
        add_kernel[blocks, threads](a_d, b_d, c_d)
    cuda.synchronize()
    kernel_time = (time.perf_counter() - start) / 100
    
    # Measure D2H transfer
    start = time.perf_counter()
    c = c_d.copy_to_host()
    cuda.synchronize()
    d2h_time = time.perf_counter() - start
    
    total = h2d_time + kernel_time + d2h_time
    
    print(f"  H2D Transfer:  {h2d_time*1000:>8.3f} ms ({h2d_time/total*100:>5.1f}%)")
    print(f"  Kernel:        {kernel_time*1000:>8.3f} ms ({kernel_time/total*100:>5.1f}%)")
    print(f"  D2H Transfer:  {d2h_time*1000:>8.3f} ms ({d2h_time/total*100:>5.1f}%)")
    print(f"  ─────────────────────────────────────")
    print(f"  Total:         {total*1000:>8.3f} ms")
    
    if (h2d_time + d2h_time) > kernel_time:
        print(f"\n  ⚠️  Transfer time ({(h2d_time + d2h_time)*1000:.2f} ms) > Kernel time ({kernel_time*1000:.2f} ms)")
        print(f"     Consider: Keep data on GPU, batch operations!")

analyze_transfer_overhead(10_000_000)

## 🎯 Exercises

### Exercise 1: Memory Reuse Pattern
Implement a pipeline that reuses GPU memory for multiple operations without copying back to CPU between each step.

In [ ]:
# TODO Exercise 1: Memory Reuse Pattern
# Compute: result = ((a + b) * c) - d
# Do this with ONE H2D transfer and ONE D2H transfer

@cuda.jit
def add_kernel(a, b, out):
    idx = cuda.grid(1)
    if idx < out.size:
        out[idx] = a[idx] + b[idx]

@cuda.jit
def mul_kernel(a, b, out):
    idx = cuda.grid(1)
    if idx < out.size:
        out[idx] = a[idx] * b[idx]

@cuda.jit  
def sub_kernel(a, b, out):
    idx = cuda.grid(1)
    if idx < out.size:
        out[idx] = a[idx] - b[idx]

def pipeline_inefficient(a, b, c, d):
    """BAD: Transfers after each operation"""
    # TODO: This is the slow way - see how many transfers happen
    pass

def pipeline_efficient(a, b, c, d):
    """GOOD: All operations on GPU, one final transfer"""
    # TODO: Implement efficient version
    # 1. Copy all inputs to GPU
    # 2. Perform all operations
    # 3. Copy only result back
    pass

# Test your implementations
# N = 10_000_000
# a = np.random.randn(N).astype(np.float32)
# ...

### Exercise 2: Memory Bandwidth Calculation
Calculate the theoretical vs achieved memory bandwidth for a copy operation.

In [ ]:
# TODO Exercise 2: Bandwidth Measurement
@cuda.jit
def copy_kernel(src, dst):
    """Simple copy kernel - measures GPU memory bandwidth"""
    idx = cuda.grid(1)
    if idx < dst.size:
        dst[idx] = src[idx]

def measure_bandwidth(n):
    """Measure achieved memory bandwidth"""
    # TODO: 
    # 1. Create two device arrays
    # 2. Time the copy kernel
    # 3. Calculate bandwidth: (bytes_read + bytes_written) / time
    # 4. Compare to theoretical peak (from device properties)
    pass

# measure_bandwidth(100_000_000)

## 📝 Key Takeaways

### Memory Management Summary:

| Method | When to Use | Pros | Cons |
|--------|-------------|------|------|
| **Explicit** (`to_device`, `copy_to_host`) | Production code, max control | Fastest, predictable | More code |
| **Unified** (`managed_array`) | Prototyping, complex access patterns | Simple code | Page fault overhead |
| **Pinned** (`pinned_array`) | High-bandwidth transfers | ~2x transfer speed | Uses system RAM |

### Best Practices:

1. **Minimize transfers**: Keep data on GPU as long as possible
2. **Use pinned memory**: For frequent large transfers
3. **Batch operations**: Don't copy back between every operation
4. **Profile first**: Measure before optimizing

### Memory Bandwidth Rule of Thumb:
- PCIe 3.0 x16: ~16 GB/s
- PCIe 4.0 x16: ~32 GB/s  
- GPU Memory: 200-900 GB/s (much faster!)

---

### 📚 Next Up: Day 4 - Error Handling & Debugging
- CUDA error codes
- Debugging techniques
- Common pitfalls

---

### 🔗 Resources
- [CUDA Memory Guide](../../cuda-programming-guide/02-basics/cuda-memory.md)
- [Understanding Memory](../../cuda-programming-guide/02-basics/understanding-memory.md)